# Search Browser - Simple way to discover search properties
The *SearchBrowser* tool is a simple python module, built on top of the Refinitiv Data (RD) Library for Python, that accepts criteria allowing users to easily discover properties and values.  As we walk through some examples, you will better understand how and where this tool will help in your construction of queries and the extraction of values that will greatly accelerate the goal of retrieving desired content.

## Use Case
The simplest way to demonstrate what this module will provide is to define a simple use case that outlines the journey of what is involved in determining and discovering the appropriate search criteria and properties to solve the problem.  

For example, we'll define the following use case:

* Retrieve a list of active bonds for a specific organization, eg: Santander Bank
* For this organization, only request bonds issued within a specific country, eg: United States
* For each bond, retrieve details such as: 
  * Maturity date
  * Issue Date
  * Coupon Rate
  * Coupon Type
  * Amount Outstanding
  * Amount Issued  
  * Issuer Country

**Important Note:** I'm not interested in Perpetual Bonds - these do not have a maturity date.

The following steps are recommended:

1. Step 1 - Start with a basic query

   The *SearchBrowser* interface supports the specification of a basic query expression as a quick way to retrieve related content.  While this is a simple and convenient way to get started, you may already have a jump-start with an existing example.  Optionally, you can supply some more detailed expressions such as a filter and possibly a view.  Either way, this step is used to define a quick way to get some relevant data from the service.

2. Step 2 - Filter out unwanted data

   Once a data set has been generated, we begin to narrow down our result set by specifying filter expressions.  This step is critical as it leads to step 3 below. In some cases, we may not discover the appropriate output properties unless we properly filter our request first.  For example, in our use case, we must ensure our results do not include perpetual bonds.  If they did, we would not be able to discover a maturity date property - perpetual bonds have no maturity.

3. Step 3 - Define the properties we want to retrieve   

   Once we have defined a relevant filter expression, we can begin the process of discovering the output properties we want to capture.

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd

%run ./SearchBrowserLib.ipynb

# Default session - desktop
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fde28341310 {name='codebook'}>

In [2]:
# Uncomment the following to see a larger data set when displaying to the screen.
#pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_colwidth', 140)

#### Create a Search Browser object
The *SearchBrowser* module is a simple class that is responsible for managing the manipulation of debug and metadata information.  The goal of this module is to hold financial properties, metadata, and data values, based on search criteria, that users can observe and interrogate at any time.

In [3]:
# Create our Search browser used throughout this workbook
browser = SearchBrowser()

#### Step 1 - Start with a basic query
When using *SearchBrowser*, the goal is to feed the tool search criteria. Initially, you will likely begin with simple, google-like, query expressions.  As you become more advanced, you will move to more complicated filters and even navigators.

In [4]:
# The SearchBrowser interface provides criteria to request for data.  Let's start with a basic query...
browser.execute("Santander bonds")

(47109, 1475)

The above execution will perform the steps to query the Search services, retrieve debug and metadata information and prepare the results within the *SearchBrowser*.  The *execute()* method returns the # of hits resulting from the request and the number of properties found in the first hit, respectively.  The browser will only contain the details related to the first hit.

In [5]:
# Dump the result of the above query...
browser.df

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
0,Wert,,130361,String,True,False,False,True,False,True
1,MaturitySinkSpeed,,NONE,String,True,False,False,False,False,False
2,RCSCouponTypeGenealogy,,M:1EU\A:C1\A:25,String,True,False,True,True,True,False
3,EcbCodeDesc,,Not ECB Eligible,String,False,False,False,False,False,False
4,WorstStandardOTRSpread,,396.460657,Double,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
1470,HasEmaxx,,True,Boolean,True,False,False,False,False,False
1471,NameLength,,17,Double,True,True,True,False,False,False
1472,RCSSeniorityTypeGenealogy,,M:1GK\A:N5,String,True,True,False,False,False,False
1473,CouponRateFormatted,,10.063,Double,True,True,True,False,False,False


This table represents the complete data set related to the above query. Effectively, a combination of the metadata and debug steps mentioned above.  At this stage, you can quickly scan the output above to verify relevancy.

#### Step 2 - Filter out unwanted data
In this step, the goal is to remove unwanted hits.  For example, we are only interested in active bonds that have been issued in the United States.  In addition, our requirement is to filter out perpetual bonds. As part of this exercise, we will continually update the search criteria and likely perform multiple executions until we believe our criteria match our requirements.  By doing this, we can properly narrow down the result set.

The above execution to retrieve 'Santander bonds' will search across the entire Search content set - this is represented by the logical view called: 'SearchAll'.  Doing so, may not only return bond instruments but other data, such as bond pricing information.

In [6]:
# First, narrow down the data set by limiting the search within a specific view
browser.execute(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    query = 'Santander bonds'
)

(25359, 1475)

The above criteria limits the result to provide instruments (bond) only.  You can see how the result set has significantly dropped down to ~25,000 hits.  Let's continue.

In [7]:
# Filter out perpetual bonds.  To do this, let's see if a relevant property exists...
browser.properties("perpetual")

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
1223,IsPerpetualSecurity,,True,Boolean,True,True,False,False,False,False


The *properties()* method provides a simple way to discover relevant properties available within the browser.  By specifying case-insensitive text, we can ask the browser to return all matching properties.  Doing so, we are presented with all properties matching our input expression: 'perpetual'.  We can clearly see our match is a suitable candidate that indicates whether the bond is perpetual or not.  We can use this value to filter out those bonds that are perpetual.

In [8]:
# Update the execution to filter out perpetual bonds...
browser.execute(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,    
    query = "santander bonds",
    filter = "IsPerpetualSecurity eq false"
)

(25231, 495)

Using this same kind of technique as we did above, we can further filter those bonds that have not already matured and are active.  Let's interrogate the browser to find these properties.

**Note** As outlined within the [Common Properties section](https://developers.refinitiv.com/en/article-catalog/article/building-search-into-your-application-workflow#Common) of the Search article, two useful properties, *IsActive* and *AssetState* can be used to help ensure the bonds we request for are active and valid.

In [9]:
# Let's ensure we have an 'active' property...
browser.properties("active")

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
449,IsActive,,True,Boolean,True,True,True,False,False,False


In [10]:
# As well as a 'status' property...
browser.properties("status")

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
229,AssetStatusEffDate,,2021-07-28T00:00:00.000Z,Date,True,True,True,False,False,False
269,AssetStatus,,ISS,String,True,False,False,False,False,False
459,AssetStatusDescription,,Issued,String,True,True,True,True,True,False


As the Search Article outlines, in some cases, the status property, *AssetStatus*, may not be available.  However, we have both.  At this stage, it is up to you which one may be more relevant.  Typically, the AssetStatus does provide more granularity about the status of the instrument, and that you can use this one as opposed to relying on the boolean state of *IsActive*.  That being said, using both may be redundant but certainly doesn't hurt.  The filter expression utilizing these 2 properties looks like this:

```
"IsActive eq True and not(AssetStatus in ('MAT' 'DC'))"
```

The above expression filters bonds that are active with a status that is neither matured nor de-activated.

In [11]:
# Next, how do we figure out bonds that are issued within a specific country?  
# Let's first see if there is a property that will help.
browser.properties("issuercountry")

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
19,RCSIssuerCountryLeafML,,ja ケイマン諸島|zh-Hans 开曼群岛|zh-Hant 開曼群島|es Islas Caimán|fr Îles Caïmans|ko 케이맨 제도|ru Каймановы острова,String,True,False,False,False,True,False
171,RCSIssuerCountryName,,Continent / Region\Americas\South America / Central America\Caribbean\Cayman Islands|Country\Cayman Islands|FRUC Regions\Global\Latin Am...,String,True,False,False,False,False,False
188,RCSIssuerCountryGenealogy,,M:DH\G:4\G:G\G:R\G:4J|M:A9\G:4J|M:1M2\G:3U8\G:3UA\G:4J,String,True,True,True,True,True,False
234,IssuerCountryContinentGroup,,Americas,String,True,False,False,False,False,False
256,IssuerCountry,,KY,String,True,True,False,False,False,False
280,RCSIssuerCountryNameML,,es Continentes/regiones\Continente Americano\Sudamérica/Centroamérica\Caribe\Islas Caimán|fr Continents/Régions\Amérique\Amérique du Sud...,String,True,False,False,False,True,False
284,RCSIssuerCountryLeaf,,Cayman Islands,String,True,True,True,True,True,False
319,RCSIssuerCountry,,G:4J,String,True,False,True,True,True,False


Based on our interrogation, we see a number of candidates that can help. For example, the RCS-based properties presented above will allow us to narrow down which country the bond was issued.  However, if we choose the property *RCSIssuerCountryLeaf*, we need to make sure the spelling is accurate.  For example, we are interested in bonds issued in the U.S.  How do we figure out the filter expression?  That is, do we look for 'USA'?  'United States'? or 'US'?  One way may that may work is to update our criteria to search for 'Santander bonds USA'.  

However, there is a better way - **Navigators**.

Navigators provide the ability to summarize the distribution of your results.

In [12]:
# The RCSIssuerCountryLeaf, or RCSIssuerCountry, properties are navigable.  
# Let's perform an execution and show what we can do...
browser.execute(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    query = "Santander bonds",
    filter = "IsPerpetualSecurity ne true and IsActive eq true and not(AssetStatus in ('MAT' 'DC'))",
    navigators = "RCSIssuerCountry, RCSIssuerCountryLeaf"
)

(1340, 495)

In [13]:
# Display the result of the above execution showing the results of the navigation...
browser.navigator

,RCSIssuerCountry,RCSIssuerCountryLeaf,Count
0,G:55,Spain,557
1,G:7J,United Kingdom,239
2,G:4M,Chile,170
3,G:4J,Cayman Islands,134
4,G:26,Brazil,70
5,G:2S,Colombia,53
6,G:6J,United States,36
7,G:3N,Norway,23
8,G:2V,Mexico,19
9,G:3D,Germany,13


Not only do we get a list of all countries within our results, but a distribution of hits for each.  For our use case, we can update the filter expression to ensure we are listing bonds that are issued within the United States.  At this point, it is worth noting the *exact* attribute value associated with the property *RCSIssuerCountryLeaf*.  A value of *True* means we can search for countries that exactly match our expression.

At this point, we should have a fairly accurate request that will pull down the list of relevant bonds.

In [14]:
# Add our country filter using an exact match expression...
browser.execute(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    query = "Santander bonds",
    filter = "IsPerpetualSecurity ne true and IsActive eq true and \
              not(AssetStatus in ('MAT' 'DC')) and RCSIssuerCountryLeaf xeq 'United States'",
)

(36, 756)

#### Step 3 - Define the properties we want to retrieve

Now that we have the set of bonds we want to capture, let's go through the action of determining the collection of properties, based on our use case.  The steps involved here to interrogate the service are the same as we outlined in Step 2 above.

Just as a reminder, we want to capture the following:
* Maturity date
* Issue Date
* Coupon Rate
* Coupon Type
* Amount Outstanding
* Amount Issued

In [15]:
# Locate Maturity Date...
browser.properties('maturitydate')

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
289,MaturityDate,,2023-01-16T00:00:00.000Z,Date,True,True,True,False,False,False


In [16]:
# Issue date...
browser.properties('issuedate')

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
314,IssueDate,,2019-07-15T00:00:00.000Z,Date,True,True,True,False,False,False


In [17]:
# Coupon rate...
browser.properties('couponrate')

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
311,CouponRate,,1.34125,Double,True,True,True,False,False,False
347,CouponRatePercent,,1.341%,String,False,False,False,False,False,False
754,CouponRateFormatted,,1.341,Double,True,True,True,False,False,False


In [18]:
# Coupon type...
browser.properties('coupontype')

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
1,RCSCouponTypeGenealogy,,M:1EU\A:1V\A:CU,String,True,False,True,True,True,False
137,RCSCouponTypeNameML,,ja クーポン タイプ\変動クーポン\その他 / コンプレックス フローター|zh-Hans 票息类型\浮动票息\其它/复杂的浮动票息|zh-Hant 票息類型\浮動票息\其它/複雜的浮動票息,String,True,False,False,False,True,False
157,RCSCouponTypeName,,Coupon Type\Floating Coupon\Other / Complex Floating Rate,String,True,True,True,True,True,False
192,CouponTypeDescription,,Floating: Floating,String,True,True,True,True,True,False
235,CouponClassX1XCouponType,,FLT|FROT,,,,,,,
414,CouponType,,FROT,String,True,True,False,False,False,False
424,RCSCouponTypeLeafML,,ja その他 / コンプレックス フローター|zh-Hans 其它/复杂的浮动票息|zh-Hant 其它/複雜的浮動票息,String,True,False,False,False,True,False
474,RCSCouponTypeLeaf,,Other / Complex Floating Rate,String,True,True,True,True,True,False
480,RCSCouponType,,A:CU,String,True,True,True,True,True,False


In [19]:
# Amount outstanding...
browser.properties('outstanding')

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
304,FaceOutstandingUSD,,720972000,Double,True,True,True,False,False,False
312,FaceOutstandingDate,,2019-07-15T00:00:00.000Z,Date,True,True,True,False,False,False
464,EOMAmountOutstanding,,720972000,Double,True,True,True,False,False,False
469,FaceOutstanding,,720972000,Double,True,True,True,False,False,False


In [20]:
# Amount issued...
browser.properties("issued")

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
169,IssuedAs,,4 Years,String,True,True,True,True,True,False
314,IssueDate,,2019-07-15T00:00:00.000Z,Date,True,True,True,False,False,False
337,FaceIssuedUSD,,720972000,Double,True,True,True,False,False,False
421,FaceIssuedTotal,,720972000,Double,True,True,True,False,False,False
443,MarketOfIssueDescription,,Eurobond,String,True,True,False,False,False,False
658,IssuedAsDays,,1281,Double,True,True,True,False,False,False


#### Putting it altogether
While going through the above exercise, I was able to successfully locate the relevant required properties.  However, this exercise will require some experimentation.  The biggest challenge with Search is how to figure out the names of the properties.  Given there are hundreds available, having the ability to quickly search for them will be invaluable.  Because of this, you will likely play with the expressions you type.  The nice thing is that you can at least see the values associated with the properties displayed in order for you to better understand the meaning behind the property.

Now that we've retrieved our output fields, let's create our final search and show the results, ordered by the date each matures.

In [21]:
response=search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    query = "Santander bonds",
    top = 100,
    filter = "IsPerpetualSecurity ne true and IsActive eq true and \
              not(AssetStatus in ('MAT' 'DC')) and RCSIssuerCountryLeaf xeq 'United States'",
    select = "MaturityDate, IssueDate, CouponRate, FaceOutstanding, FaceIssuedTotal, RCSCouponTypeLeaf, ISIN",
    order_by = "MaturityDate"
).get_data()
response.data.df

,MaturityDate,IssueDate,CouponRate,FaceOutstanding,FaceIssuedTotal,RCSCouponTypeLeaf,ISIN
0,2022-05-02T00:00:00.000Z,2022-02-02T00:00:00.000Z,0.32,<NA>,<NA>,Pay at Maturity Fixed,US05966DE656
1,2022-05-09T00:00:00.000Z,2022-02-08T00:00:00.000Z,0.38,<NA>,<NA>,Pay at Maturity Fixed,US05966DF232
2,2022-05-10T00:00:00.000Z,2022-03-11T00:00:00.000Z,0.815,<NA>,<NA>,Pay at Maturity Fixed,US05966DF729
3,2022-05-13T00:00:00.000Z,2022-02-14T00:00:00.000Z,0.51,<NA>,<NA>,Pay at Maturity Fixed,US05966DF315
4,2022-05-18T00:00:00.000Z,2021-11-04T00:00:00.000Z,0.25,<NA>,<NA>,Pay at Maturity Fixed,US05966DD666
5,2022-05-20T00:00:00.000Z,2021-11-08T00:00:00.000Z,0.245,<NA>,<NA>,Pay at Maturity Fixed,US05966DD740
6,2022-05-23T00:00:00.000Z,2022-03-11T00:00:00.000Z,0.85,<NA>,<NA>,Pay at Maturity Fixed,US05966DF646
7,2022-05-27T00:00:00.000Z,2022-01-28T00:00:00.000Z,0.42,<NA>,<NA>,Pay at Maturity Fixed,US05966DE573
8,2022-06-02T00:00:00.000Z,2022-03-02T00:00:00.000Z,0.7,<NA>,<NA>,Pay at Maturity Fixed,US05966DF562
9,2022-06-08T00:00:00.000Z,2021-06-08T00:00:00.000Z,0.21,<NA>,<NA>,Pay at Maturity Fixed,US05966DB926


## Browser Features
The *SearchBrowser* was designed to facilitate many useful interrogation features enabling the user to rapidly discover properties. Given the power and flexibility of the Search service, you may need multiple ways to arrive at your discovery.  For example, you may generally know, or guess, at the name of a property, or you may know a value to look for.  If you are in a position where the name or value does not provide the answers, you may need to narrow down your results based on the property type or property that provides navigation.  The bottom line is that the more tools you have available for use, the better the success rate at building your expressions.

**Note**: As you go through the exercise of locating properties, you may notice there are no relevant candidates for you to choose from, despite all the tools available within the *SearchBrowser*.  If you are unable to figure out why certain properties are unavailable for the kind of data you are retrieving, this may be related to the search criteria provided.  Because the *SearchBrowser* only captures data for the first hit encountered, updating your search criteria may present a different data set and as a result, different properties.  Otherwise, you may need to reach out to the Refinitiv Helpdesk and they can involve a Content Specialist who can investigate further.

Below is a general outline of the core features.  For each, I have provided some context when you may need to use them to give you a better understanding.

#### Properties

In [22]:
help(SearchBrowser.properties)

Help on function properties in module __main__:

properties(self, text)
    Browse the properties that match the text expression.
    Eg: browser.properties('ISIN')



#### Values

In [23]:
help(SearchBrowser.values)

Help on function values in module __main__:

values(self, text)
    Browse the values that match the text expression.
    Eg: browser.values('united kingdom') - returns all values containing the expression 'united kingdom'
        browser.values(238)   - returns all numeric rows with an exact match
        browser.values(2.345)



While the *properties()* method is extremely useful, at times you may be using the desktop, or some other tool, to build out your results and have the need to convert your request using the Search API.  For example, in the above Use Case, we used the query expression **"Santander bonds"** to pull bonds for a specific company.  In most cases, using a company name or ticker will generate an accurate list of bonds. However, there may be instances where Search will associate a specific query belonging to multiple organizations, thus generating an invalid list of bonds. This is entirely dependent on your query expression and that Search performs its queries that closely match your expression, as opposed to finding only hits related to the company expression you provided.

Alternatively, you may prefer to use the Company ID (*Organization ID/Perm ID*).  In this scenario, if you have used the desktop to retrieve the Company Perm ID, eg: *8589934205*, you can use this value to build out a filter that returns hits only related to this specific company ID.

In [24]:
# List all properties that contain the following value...
browser.values("8589934205")

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
305,ParentOAPermID,,8589934205,String,True,False,False,True,True,True
395,ParentImmedOAPermID,,8589934205,String,True,False,False,True,True,True
446,HierarchyOAPermIDs,,5000001412|8589934205,String,True,False,False,True,True,True


Using the above results, I can update my request by adding the following criteria within our filter:

```
"ParentOAPermID xeq '8589934205'"
```

In [25]:
# Remove query and replace with an updated filter expression
browser.execute(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "ParentOAPermID xeq '8589934205' and IsPerpetualSecurity ne true and IsActive eq true and \
              not(AssetStatus in ('MAT' 'DC')) and RCSIssuerCountryLeaf xeq 'United States'",
)

(50, 626)

Applying the new filter, you may notice the number of hits may be different. 

**Note**: Listed bonds change daily and the results may vary.

To justify why this may occur, will require a deeper analysis of the data.  This is where the SearchBrowser shines.  Using the available tools will allow users to easily interrogate fields and values to understand what conditions may have caused the disparity.  To be successful, users will require some domain knowledge of the data they are retrieving.  While the disparity above is beyond the scope of this article and may require a content specialist to investigate, users with some domain knowledge can interrogate the results, looking for conditions or settings that provide an explanation.

#### Navigators

In [26]:
help(SearchBrowser.navigable)

Help on function navigable in module __main__:

navigable(self, prop=None, value=None)
    Browse the metadata that matches all properties that are navigable.
    Apply additional criteria that matches properties or values.
    Eg: browser.navigable()              - returns all navigable properties
        browser.navigable('Description') - returns all navigable properties containing 'Description'
        browser.navigable(value='euro')  - returns all navigable properties with a value containing 'euro'
        browser.navigable('RCS', 'euro') - returns all RCS-based navigable properties with a value containing 'euro'



When you begin your journey to hunt down properties, you will notice a Navigable attribute as part of the output listed for each property. A value of true indicates that the property provides a well-defined bucket of values. For example, an industry sector, asset category, or country code is a typical navigator that defines a collection of values associated with that property. Navigators not only bucket data for users to understand the domain of how data is collected but include distribution of how many hits are specific to your search criteria.

Using navigators, I can discover an appropriate property.

For example, within our specific use case, you will notice many of the bonds returned do not contain values for "FaceOutstanding" or "FaceIssuedTotal".  Doing some more investigated work with a content specialist, I discovered that this data is not collected/available for bonds that belong to the "certificates of deposits" category.  If I prefer to filter out bonds that belong to this category, I can approach this in a few ways.  One simple way is to utilize the *properties()* feature to search for anything related to a 'category'.  However, it may be more worthwhile to determine if there are other categories I may want to filter out.

As a first step, let's determine if there is a navigable category property:

In [27]:
# Give me all navigators that may related to a category...
browser.navigable('category')

,Property,Nested,Value,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
95,SearchAllCategoryv3,,Bonds,String,True,True,True,True,True,False
104,RCSAssetCategory,,A:14,String,True,False,True,True,True,False
128,AssetCategoryName,,Fixed Income/Interest At Maturity Bond,String,True,False,True,False,True,False
153,RCSAssetCategoryGenealogy,,A:3\A:8\A:14,String,True,False,True,False,True,False
215,RCSAssetCategoryName,,FX / Money / Interest Rate Derivative\Money Market\Certificate of Deposit,String,True,True,True,True,True,False
261,SearchAllCategory,,Bonds,String,True,True,True,True,True,False
354,RCSAssetCategoryLeaf,,Certificate of Deposit,String,True,True,True,True,True,False
359,AssetCategory,,IMB,String,True,False,True,False,True,False
368,TRCSAssetCategoryLeaf,,Money Market,String,True,True,True,True,True,False
371,DerivedCategory,,BOND,String,True,True,True,True,True,False


At this moment, I can see many candidates to choose from.  The one that seems most interesting/applicable is the 'AssetCategoryLeaf'.  Let's list out all the asset categories for my latest query.

In [28]:
# Select the 'RCSAssetCategory' and 'RCSAssetCategoryLeaf' to navigate.
browser.execute(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "ParentOAPermID xeq '8589934205' and IsPerpetualSecurity ne true and IsActive eq true and \
              not(AssetStatus in ('MAT' 'DC')) and RCSIssuerCountryLeaf xeq 'United States'",
    navigators = "RCSAssetCategory, RCSAssetCategoryLeaf"
)

(50, 626)

In [29]:
# Because we specified a 'navigator' as part of our execution, we can view the distribution
browser.navigator

,RCSAssetCategory,RCSAssetCategoryLeaf,Count
0,A:14,Certificate of Deposit,33
1,A:J,Bond,16
2,A:K1,Preferred Share,1


In [30]:
# Updating our search to filter out 'Certificate of Deposit'. In our case, we'll use the corresponding category code: 'A14'
# It's worth noting that I could simply ignore all categories except 'Bond' as an alternative.
# This depends on your requirements.
response=search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    top = 100,
    filter = "ParentOAPermID xeq '8589934205' and IsPerpetualSecurity ne true and IsActive eq true and \
              not(AssetStatus in ('MAT' 'DC')) and RCSIssuerCountryLeaf xeq 'United States' and \
              RCSAssetCategory ne 'A:14'",
    select = "MaturityDate, IssueDate, CouponRate, FaceOutstanding, FaceIssuedTotal, RCSCouponTypeLeaf, \
              ISIN, RCSAssetCategoryLeaf",
    order_by = "MaturityDate"
).get_data()
response.data.df

,MaturityDate,IssueDate,CouponRate,FaceOutstanding,FaceIssuedTotal,RCSCouponTypeLeaf,ISIN,RCSAssetCategoryLeaf
0,2023-01-16T00:00:00.000Z,2019-07-15T00:00:00.000Z,1.34125,720972000,720972000,Other / Complex Floating Rate,USU8029KAL62,Bond
1,2023-01-16T00:00:00.000Z,2019-07-15T00:00:00.000Z,1.34125,720972000,720972000,Other / Complex Floating Rate,US80282KBA34,Bond
2,2023-01-18T00:00:00.000Z,2017-12-18T00:00:00.000Z,3.4,750000,1000000000,Plain Vanilla Fixed Coupon,USU8029KAH50,Bond
3,2023-01-18T00:00:00.000Z,2017-12-18T00:00:00.000Z,3.4,750000,1000000000,Plain Vanilla Fixed Coupon,US80282KAQ94,Bond
4,2023-01-18T00:00:00.000Z,2018-03-07T00:00:00.000Z,3.4,999250000,999250000,Plain Vanilla Fixed Coupon,US80282KAS50,Bond
5,2024-06-07T00:00:00.000Z,2019-06-07T00:00:00.000Z,3.5,1000000000,1000000000,Plain Vanilla Fixed Coupon,US80282KAW62,Bond
6,2025-06-02T00:00:00.000Z,2020-06-01T00:00:00.000Z,3.45,1000000000,1000000000,Plain Vanilla Fixed Coupon,US80282KBB17,Bond
7,2025-07-17T00:00:00.000Z,2015-07-17T00:00:00.000Z,4.5,1100000000,1100000000,Plain Vanilla Fixed Coupon,US80282KAE64,Bond
8,2026-10-05T00:00:00.000Z,2019-10-04T00:00:00.000Z,3.244,1080000,948853000,Plain Vanilla Fixed Coupon,US80282KAY29,Bond
9,2026-10-05T00:00:00.000Z,2019-10-04T00:00:00.000Z,3.244,1080000,948853000,Plain Vanilla Fixed Coupon,USU8029KAM46,Bond


In [31]:
response=search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    top = 100,
    filter = "ParentOAPermID xeq '8589934205' and IsPerpetualSecurity ne true and IsActive eq true and \
              not(AssetStatus in ('MAT' 'DC')) and RCSIssuerCountryLeaf xeq 'United States' and \
              RCSAssetCategoryLeaf eq 'Bond'",
    select = "MaturityDate, IssueDate, CouponRate, FaceOutstanding, FaceIssuedTotal, RCSCouponTypeLeaf, \
              ISIN, RCSAssetCategoryLeaf",
    order_by = "MaturityDate"
).get_data()
response.data.df

,MaturityDate,IssueDate,CouponRate,FaceOutstanding,FaceIssuedTotal,RCSCouponTypeLeaf,ISIN,RCSAssetCategoryLeaf
0,2023-01-16T00:00:00.000Z,2019-07-15T00:00:00.000Z,1.34125,720972000,720972000,Other / Complex Floating Rate,USU8029KAL62,Bond
1,2023-01-16T00:00:00.000Z,2019-07-15T00:00:00.000Z,1.34125,720972000,720972000,Other / Complex Floating Rate,US80282KBA34,Bond
2,2023-01-18T00:00:00.000Z,2017-12-18T00:00:00.000Z,3.4,750000,1000000000,Plain Vanilla Fixed Coupon,USU8029KAH50,Bond
3,2023-01-18T00:00:00.000Z,2017-12-18T00:00:00.000Z,3.4,750000,1000000000,Plain Vanilla Fixed Coupon,US80282KAQ94,Bond
4,2023-01-18T00:00:00.000Z,2018-03-07T00:00:00.000Z,3.4,999250000,999250000,Plain Vanilla Fixed Coupon,US80282KAS50,Bond
5,2024-06-07T00:00:00.000Z,2019-06-07T00:00:00.000Z,3.5,1000000000,1000000000,Plain Vanilla Fixed Coupon,US80282KAW62,Bond
6,2025-06-02T00:00:00.000Z,2020-06-01T00:00:00.000Z,3.45,1000000000,1000000000,Plain Vanilla Fixed Coupon,US80282KBB17,Bond
7,2025-07-17T00:00:00.000Z,2015-07-17T00:00:00.000Z,4.5,1100000000,1100000000,Plain Vanilla Fixed Coupon,US80282KAE64,Bond
8,2026-10-05T00:00:00.000Z,2019-10-04T00:00:00.000Z,3.244,1080000,948853000,Plain Vanilla Fixed Coupon,US80282KAY29,Bond
9,2026-10-05T00:00:00.000Z,2019-10-04T00:00:00.000Z,3.244,1080000,948853000,Plain Vanilla Fixed Coupon,USU8029KAM46,Bond


In [32]:
rd.close_session()